# Radial Profiles of Star Clusters 
Part of Prof. Hanno Rein's ASTC02 course on Practical Astronomy. 
In this tutorial, we will read in an image and extract the radial profile of a star cluster with python. You will need to do this for your lab report. However, in addition to what is shown in this tutorial you should also:
- Use RAW files instead of a JPG image
- Do a dark and flat field correction of your image before processing it

In [ ]:
from scipy import ndimage, misc, optimize
import numpy as np
from mpldatacursor import datacursor
import matplotlib.pyplot as plt
%matplotlib nbagg

Here, we use a simple JPG image of M3 as a test case. 

In [ ]:
m3 = np.average(misc.imread('m3.jpg'),axis=2)

Note that we have have averaged the colorr components. Colour is not important for this analysis and we get a slightly better signal to noise ratio by averaging them, rather than just picking one. If you pick just one, use the red channel as our camera is most sensitive in the red part of the spectrum.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
ax.imshow(m3);
dc = datacursor(ax)

# Calibration

We need to calibrate the image. To do that we need the brightness of a reference star. Look up in stellarium which stars there are in the above image and write down their magnitude in the V band (visible light). This is not exactly the right filter for our camera, but it'll do for our purposes.

Click on one reference star above, then execute the following cell to extract the image around the star and sum up all the pixels.

In [ ]:
x, y = [int(t) for t in dc.annotations[ax].xy]
print(x,y)
m3cs1 = m3[y-50:y+50,x-50:x+50] # top left V 9.8  / bottom left V 10.5
s1 = np.sum(m3cs1)
print(s1)

fig, axn = plt.subplots(1, 1, figsize=(3, 3))
axn.imshow(m3cs1,vmin=0,vmax=260)

Repeat the above with a different reference star.

In [ ]:
x, y = [int(t) for t in dc.annotations[ax].xy]
print(x,y)
m3cs2 = m3[y-50:y+50,x-50:x+50] # top left V 9.8  / bottom left V 10.5
s2 = np.sum(m3cs2)
print(s2)

fig, axn = plt.subplots(1, 1, figsize=(3, 3))
axn.imshow(m3cs2,vmin=0,vmax=260)

Calculate the relative brightness of the two stars relative to each toher. To convert the sum of all the pixel values to a magnitude, you need to take the log and normalize it with respect to 2.512 (the magic number in astornomy, which is purely historical, but apparently is somehow related to 

In [ ]:
np.log(s1/s2)/np.log(2.512) # should be 10.5-9.8 = 0.7

# Brightness of the cluster

Click roughly on the centre of the star cluster to get the coordinates.

In [ ]:
x, y = [int(t) for t in dc.annotations[ax].xy]
m3c = m3[y-500:y+500,x-500:x+500] 
s3 = np.sum(m3c)

fig, ax3 = plt.subplots(1, 1, figsize=(4, 4))
ax3.imshow(m3c,vmin=0,vmax=260)

In [ ]:
magV = np.log(s1/s3)/np.log(2.512)+10.5 ## should be V 6.2
magV

# Luminosity

We can calculate the absolute luminosity if we know the distance. It's hard to measure the distance, so for this course it's ok to look it up. We want to express the absolute brightness in units of the Sun's absolute brightness. The relative brightness of the Sun in the V band is -26.74.

First, let's calculate the flux ratio.

In [ ]:
fluxratio = 2.512**(-26.74-magV)
fluxratio

Next, I give you the distance to this cluster in astronomical units. This is the distance ratio.

In [ ]:
distance = 10.4e3*206264.81 # 10.4kpc in au
distance

In [ ]:
luminosityratio = fluxratio*distance**2
luminosityratio  # approx number of stars. should be 500000

# Plummer Model and Radial Profile
We want to plot a Plummer Model for the star cluster. It has the functional form:
$$\rho(r) = \frac{3M}{4 \pi a^3} \left(1+\frac{r^2}{a^2}\right)^{-5/2}$$
$$\Sigma(r) = \frac{M}{\pi a^2}\frac1{\left(1+\frac{r^2}{a^2}\right)^2}$$
The first step is to fint the centre of the cluster.

In [ ]:
xl = np.linspace(0,m3c.shape[0]-1,m3c.shape[0])
yl = np.linspace(0,m3c.shape[1]-1,m3c.shape[1])
xx, yy = np.meshgrid(yl,xl)

In [ ]:
cx = np.sum(xx*m3c)/np.sum(m3c)
cy = np.sum(yy*m3c)/np.sum(m3c)
cx,cy

In [ ]:
fig, ax4 = plt.subplots(1, 1, figsize=(4, 4))
ax4.imshow(m3c,vmin=0,vmax=255)
ax4.plot(cx, cy, 'r+')

Next, we create radial bins and sum of the light contribution in each bin.

In [ ]:
rr = np.sqrt(np.power(xx-cx,2) + np.power(yy-cy,2))

In [ ]:
rbins = np.linspace(0,500,500)
dbins = np.linspace(0,500,500)
nbins = np.linspace(0,500,500)
rf, mf = rr.flatten(), m3c.flatten()
for j in range(len(mf)):
    i = int(rf[j])
    if i<500:
        dbins[i] += mf[j]
        nbins[i] += 1

In [ ]:
def Sigma(r,m,a):
    return m/(np.pi*a**2)/(1.+(rbins/a)**2)**2
fig, ax5 = plt.subplots(1, 1, figsize=(4, 4))
ax5.plot(rbins,dbins/nbins)
m = s3
a = 1800 # fit a!
ax5.plot(rbins,Sigma(rbins,m,a))

The half mass radius is defined as $1.3*a$.

In [ ]:
rh = 1.3 * a # in pixels for now

Next, we use the focal length of telescope (1600mm), the size of the sensor (22.2 mm × 14.8 mm), and the number of pixels on our sensor (4,272 × 2,848) to determine the angular size corresponding to one pixel.

In [ ]:
#size of 1 pixel
sp1, sp2 = 22.2/4272, 14.8/2848
sp1,sp2

In [ ]:
# angular resolution of 1 pixel (radian)
ar = 2*np.arctan(sp1/(2.*1600.))
ar

In [ ]:
sizeofM3 = rh*ar*180/np.pi*60 # in arcmin
sizeofM3  # should be 2.31 arcmin 

In [ ]:
sizeofM3pc = rh*ar*10.4e3  # in parsec, using small angle approximation (which is really good here!)
sizeofM3pc